In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [44]:
# download dataset

# !wget https://data.cityofnewyork.us/api/views/tqtj-sjs8/rows.csv?accessType=DOWNLOAD
# !wget https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD&bom=true&query=select+*
# !wget https://data.cityofnewyork.us/api/views/tqtj-sjs8/rows.csv?accessType=DOWNLOAD

In [45]:
# motor_df = pd.read_csv('./data/Motor_Vehicle_Collisions_-_Person.csv')

In [46]:
# motor_df.head()

In [47]:
# motor_df['PED_LOCATION'].unique()

In [48]:
# missingValue = motor_df.isna().sum().to_frame(name='NaN_number')
# missingValue.sort_values('NaN_number',inplace=True,ascending=False)
# missingValue.plot.bar(color='blue',figsize=(15,7))
# plt.title('Missing Value of Motor Vehicle Collisions dataset',fontsize=15)
# plt.legend()
# plt.show()

In [8]:
crash_df = pd.read_csv('./data/Motor_Vehicle_Collisions_-_Crashes.csv',dtype={'BOROUGH':str,'ZIP CODE':str})

In [6]:
# crash_df.columns.unique().to_list()

In [9]:
crash_df['CRASH DATE'] = pd.to_datetime(crash_df['CRASH DATE'])
crash_df['CRASH DATE'].sort_values(ascending=False)
crash_df['dow'] = pd.to_datetime(crash_df['CRASH DATE']).dt.dayofweek
crash_df['day'] = pd.to_datetime(crash_df['CRASH DATE']).dt.day
crash_df['year'] = pd.to_datetime(crash_df['CRASH DATE']).dt.year

In [10]:
crash_df['Killed'] = crash_df['NUMBER OF PERSONS KILLED']+\
                        crash_df['NUMBER OF PEDESTRIANS KILLED']+crash_df['NUMBER OF CYCLIST KILLED']
crash_df['Injured'] = crash_df['NUMBER OF PERSONS INJURED']+\
                        crash_df['NUMBER OF PEDESTRIANS INJURED']+crash_df['NUMBER OF CYCLIST INJURED']

In [21]:
crash_df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,dow,day,year,Killed,Injured
0,2020-03-13,11:00,NaN,NaN,40.655518,-74.00644,"(40.655518, -74.00644)",3 AVENUE,NaN,NaN,...,Station Wagon/Sport Utility Vehicle,Tractor Truck Diesel,NaN,NaN,NaN,4,13,2020,0.0,0.0
1,2020-05-19,12:04,NaN,NaN,40.603070,-74.01667,"(40.60307, -74.01667)",BELT PARKWAY,NaN,NaN,...,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,1,19,2020,0.0,3.0
2,2020-04-09,23:45,BROOKLYN,11207,40.675526,-73.90132,"(40.675526, -73.90132)",NaN,NaN,2517 ATLANTIC AVENUE,...,Sedan,Sedan,NaN,NaN,NaN,3,9,2020,0.0,2.0
3,2020-05-26,0:53,STATEN ISLAND,10306,40.574898,-74.09110,"(40.574898, -74.0911)",OLYMPIA BOULEVARD,HUNTER AVENUE,NaN,...,Station Wagon/Sport Utility Vehicle,Pick-up Truck,NaN,NaN,NaN,1,26,2020,0.0,0.0
4,2020-03-14,15:30,BROOKLYN,11234,40.625633,-73.90917,"(40.625633, -73.90917)",BERGEN AVENUE,ROYCE PLACE,NaN,...,Pick-up Truck,NaN,NaN,NaN,NaN,5,14,2020,0.0,0.0


In [20]:
crash_df.isna().sum()

CRASH DATE                             0
CRASH TIME                             0
BOROUGH                           541636
ZIP CODE                          541847
LATITUDE                          208858
LONGITUDE                         208858
LOCATION                          208858
ON STREET NAME                    355423
CROSS STREET NAME                 620171
OFF STREET NAME                  1500153
NUMBER OF PERSONS INJURED             18
NUMBER OF PERSONS KILLED              31
NUMBER OF PEDESTRIANS INJURED          0
NUMBER OF PEDESTRIANS KILLED           0
NUMBER OF CYCLIST INJURED              0
NUMBER OF CYCLIST KILLED               0
NUMBER OF MOTORIST INJURED             0
NUMBER OF MOTORIST KILLED              0
CONTRIBUTING FACTOR VEHICLE 1       4960
CONTRIBUTING FACTOR VEHICLE 2     249675
CONTRIBUTING FACTOR VEHICLE 3    1645038
CONTRIBUTING FACTOR VEHICLE 4    1737783
CONTRIBUTING FACTOR VEHICLE 5    1756567
COLLISION_ID                           0
VEHICLE TYPE COD

In [36]:
# TODO: Use latitude and longitude to get the borough name
crash = crash_df[crash_df['LATITUDE'].notna()].copy()
crash.isna().sum()

CRASH DATE                             0
CRASH TIME                             0
BOROUGH                           364392
ZIP CODE                          364586
LATITUDE                               0
LONGITUDE                              0
LOCATION                               0
ON STREET NAME                    302160
CROSS STREET NAME                 541378
OFF STREET NAME                  1314233
NUMBER OF PERSONS INJURED             16
NUMBER OF PERSONS KILLED              28
NUMBER OF PEDESTRIANS INJURED          0
NUMBER OF PEDESTRIANS KILLED           0
NUMBER OF CYCLIST INJURED              0
NUMBER OF CYCLIST KILLED               0
NUMBER OF MOTORIST INJURED             0
NUMBER OF MOTORIST KILLED              0
CONTRIBUTING FACTOR VEHICLE 1       4455
CONTRIBUTING FACTOR VEHICLE 2     224480
CONTRIBUTING FACTOR VEHICLE 3    1450701
CONTRIBUTING FACTOR VEHICLE 4    1531858
CONTRIBUTING FACTOR VEHICLE 5    1548388
COLLISION_ID                           0
VEHICLE TYPE COD

In [34]:
# crash_year = crash_df[['year','COLLISION_ID','Killed','Injured']]\
#                 .groupby('year').agg({'COLLISION_ID':'count','Killed':'sum','Injured':'sum'})\
#                 .reset_index()\
#                 .rename(columns={'COLLISION_ID':'Number of Collsion'})
# crash_year = crash_year[crash_year['year']!=2021]

In [58]:
# plt.figure(figsize=(10,5))
# plt.bar(crash_year.year,crash_year['Number of Collsion'],color='lightblue')
# plt.title('Number of Collisions from 2012 to 2020',fontsize=20,color='white')
# plt.ylabel('Collisions',fontsize=15,color='white')
# plt.xticks(crash_year.year,fontsize=13,color='white')
# plt.yticks(fontsize=13,color='white')
# plt.grid(linestyle='--',axis='y')
# plt.savefig('collisions',dpi=300)
# plt.show()

In [59]:
# plt.figure(figsize=(10,5))
# plt.bar(crash_year.year,crash_year.Injured,color='lightblue')
# plt.title('Number of Injuries from 2012 to 2020',fontsize=20,color='white')
# plt.ylabel('Injuries',fontsize=15,color='white')
# plt.xticks(crash_year.year,fontsize=13,color='white')
# plt.yticks(fontsize=13,color='white')
# plt.grid(linestyle='--',axis='y')
# plt.savefig('injuries',dpi=300)
# plt.show()

In [60]:
# plt.figure(figsize=(10,5))
# plt.bar(crash_year.year,crash_year.Killed,color='lightblue')
# plt.title('Number of Fatalities from 2012 to 2020',fontsize=20,color='white')
# plt.ylabel('Fatalities',fontsize=15,color='white')
# plt.xticks(crash_year.year,fontsize=13,color='white')
# plt.yticks(fontsize=13,color='white')
# plt.grid(linestyle='--',axis='y')
# plt.savefig('fatalities',dpi=300)
# plt.show()

In [14]:
# missing_value_crash = crash_df.isna().sum().to_frame(name='NaN_number')
# missing_value_crash.sort_values('NaN_number',inplace=True,ascending=False)
# missing_value_crash.plot.bar(color='blue',figsize=(15,7))
# plt.title('Missing Value of Motor Vehicle Collisions dataset',fontsize=15)
# plt.legend()
# plt.show()

In [22]:
permit_df = pd.read_csv('./data/Street_Construction_Permits.csv', low_memory=True)

/home/valx/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3,4,8,14,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [24]:
permit_df.shape

(10407640, 36)

In [23]:
permit_df.isna().sum()

PermitNumber                          0
ApplicationTrackingID           6150227
SequenceNumber                  5579305
ApplicationTypeShortDesc        6150227
PermitStatusID                      229
PermitStatusShortDesc               229
PermitSeriesID                        0
PermitSeriesShortDesc                 0
PermitTypeID                          0
PermitTypeDesc                        0
PermitNumberOfZones             3722517
PermitLinearFeet                5425553
PermitTotalSqFeet               8193368
PermitEstimatedNumberOfCuts     6834771
EquipmentTypeDesc               9798652
NumberOfContainers             10292775
NumberOfMiniContainers         10339616
SpecificStipulations            4461181
PreviousPermitNumber            7199019
NextPermitNumber                7600226
EmergencyIssueDate              9894586
PermitIssueDate                   21802
IssuedWorkStartDate               46342
IssuedWorkEndDate                 59682
BoroughName                      290964


In [31]:
permit = permit_df[(permit_df['BoroughName'].notna()) & (permit_df['OnStreetName'].notna())\
                & (permit_df['IssuedWorkStartDate'].notna()) & (permit_df['IssuedWorkEndDate'].notna())].copy()

In [33]:
permit.shape

(10096248, 36)

In [43]:
# permit_df.columns.to_list()

In [42]:
# permit_df['EmergencyIssueDate']

In [41]:
# permit_df['CreatedOn'].str[0:10]

In [33]:
# permit_df['CreatedOn'] = pd.to_datetime(permit_df['CreatedOn'].str[:10])

In [40]:
# permit_df['CreatedOn'].sort_values(ascending=True)

In [35]:
# permit_df['dow'] = pd.to_datetime(permit_df['CreatedOn']).dt.dayofweek
# permit_df['day'] = pd.to_datetime(permit_df['CreatedOn']).dt.day
# permit_df['year'] = pd.to_datetime(permit_df['CreatedOn']).dt.year

In [39]:
# permit_year = permit_df[['year','PermitNumber']].groupby('year').count().reset_index()
# permit_year.head()

In [38]:
# plt.figure(figsize=(10,5))
# plt.bar(crash_year.year,crash_year.Injured,color='lightblue')
# plt.title('Number of Injuries from 2012 to 2020',fontsize=20,color='white')
# plt.ylabel('Injuries',fontsize=15,color='white')
# plt.xticks(crash_year.year,fontsize=13,color='white')
# plt.yticks(fontsize=13,color='white')
# plt.grid(linestyle='--',axis='y')
# plt.savefig('injuries',dpi=300)
# plt.show()

In [38]:
# missing_value_permit = permit_df.isna().sum().to_frame(name='NaN_number')
# missing_value_permit.sort_values('NaN_number',inplace=True,ascending=False)

In [37]:
# missing_value_permit.plot.bar(color='blue',figsize=(15,7))
# plt.title('Missing Value of Street Construction Permit dataset',fontsize=15)
# plt.legend()
# plt.show()

In [40]:
# import pandas as pd
# import numpy as np
# #import geopandas as gpd
# import dask.dataframe as dd
# from dask.diagnostics import ProgressBar
# from datetime import datetime
# from matplotlib import pyplot as plt
# import warnings
# %matplotlib inline
# warnings.filterwarnings('ignore')

# permit_dd = dd.read_csv('./data/Street_Construction_Permits.csv',dtype=str)
# permit_dd['CreatedOn'] = dd.to_datetime(permit_dd['CreatedOn'].str[:10])

# permit_dd['dow'] = dd.to_datetime(permit_dd['CreatedOn']).dt.dayofweek
# permit_dd['day'] = dd.to_datetime(permit_dd['CreatedOn']).dt.day
# permit_dd['year'] = dd.to_datetime(permit_dd['CreatedOn']).dt.year

# permit_year = permit_dd[['year','PermitNumber']].groupby('year').count().reset_index().compute()
# permit_year['year']= permit_year[permit_year['year'] != '2021']
# permit_year = permit_year[0:-1]
# permit_year.year=permit_year['year'].astype(str)
# permit_year.year = permit_year.year.str[:4]
# permit_year = permit_year[1:].sort_values('year')

# permit = permit_dd.compute()

In [61]:
# plt.figure(figsize=(10,5))
# plt.bar(permit_year.year,permit_year['PermitNumber'],color='lightblue')
# plt.title('Number of Construction Permits from 2014 to 2020',fontsize=20,color='white')
# plt.ylabel('Number of Permits',fontsize=15,color='white')
# plt.xticks(permit_year.year,fontsize=13,color='white')
# plt.yticks(fontsize=13,color='white')
# plt.grid(linestyle='--',axis='y')
# plt.savefig('permits',dpi=300)
# plt.show()

In [57]:
# permit_year['PermitNumber'].sum()/len(permit_year['PermitNumber'])

628370.0